In [1]:
# Dependencies
import os
import pandas as pd
from bs4 import BeautifulSoup as bs
import requests
import pymongo
from splinter import Browser
from selenium import webdriver
from splinter.exceptions import ElementDoesNotExist
import time
from time import sleep

In [2]:
executable_path = {"executable_path": 'chromedriver'}
browser = Browser("chrome", **executable_path, headless=False)

## NASA Mars News

In [3]:
url = 'https://mars.nasa.gov/news/'
browser.visit(url)
# bottom image load first in the same div, then these react text ul class= "item_list" and li class+"slide" load so we need a way to pause until the whole page is loaded
browser.is_element_present_by_css("ul.item_list li.slide", wait_time=1)
# use splinter to grab html
html = browser.html
soup = bs(html, 'html.parser')

In [4]:
#print(soup.prettify())

In [5]:
# Scrape the [NASA Mars News Site](https://mars.nasa.gov/news/) and collect the latest News Title and Paragraph Text. Assign the text to variables that you can reference later.
news_title = soup.find('div', class_="content_title").get_text()
news_paragraph = soup.find('div', class_="article_teaser_body").get_text()

print(news_title)
print(news_paragraph)

A Rover Pit Stop at JPL
Working like a finely honed machine, a team of engineers in this time-lapse video clip install test wheels on another finely honed machine: NASA's Mars 2020 rover.


## JPL Space Featured Image

In [6]:
# Visit the url for JPL Featured Space Image [here](https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars).
# Use splinter to navigate the site and find the image url for the current Featured Mars Image and assign the url string to a variable called `featured_image_url`.
# Make sure to find the image url to the full size `.jpg` image.
# Make sure to save a complete url string for this image.

jpl_url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(jpl_url)
#jpl_html = browser.html
#jpl_soup = bs(jpl_html, "lxml")

In [7]:
# splinter click first 'full image' button
full_image_elem = browser.find_by_id('full_image')
full_image_elem.click()

In [8]:
## splinter click first 'more info' button
browser.is_element_present_by_text('more info   ', wait_time=2)
more_info_elem = browser.find_link_by_partial_text('more info')

In [9]:
more_info_elem.click()

In [10]:
jpl_html = browser.html
jpl_soup = bs(jpl_html, "lxml")

In [11]:
img_url_rel = jpl_soup.select_one('figure.lede a img').get("src")
img_url_rel

'/spaceimages/images/largesize/PIA18049_hires.jpg'

In [12]:
featured_image_url = f'https://www.jpl.nasa.gov{img_url_rel}'
print(str(featured_image_url))

https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA18049_hires.jpg


## Mars Twitter

In [13]:
# * Visit the Mars Weather twitter account [here](https://twitter.com/marswxreport?lang=en) and scrape the latest Mars weather tweet from the page. Save the tweet text for the weather report as a variable called `mars_weather`.
twitter_url = 'https://twitter.com/marswxreport?lang=en'
browser.visit(twitter_url)
twitter_html = browser.html
twitter_soup = bs(twitter_html, "lxml")

In [14]:
# find the lastest tweet con't
latest_weather = twitter_soup.find('div', class_='js-tweet-text-container')
print(latest_weather)

<div class="js-tweet-text-container">
<p class="TweetTextSize TweetTextSize--normal js-tweet-text tweet-text" data-aria-label-part="0" lang="en">The <a class="twitter-hashtag pretty-link js-nav" data-query-source="hashtag_click" dir="ltr" href="/hashtag/Mars2020?src=hash"><s>#</s><b>Mars2020</b></a> rover gets her wheels from the pit crew with PhDs
<a class="twitter-timeline-link" data-expanded-url="https://mars.nasa.gov/mars2020/mission/where-is-the-rover/" dir="ltr" href="https://t.co/525ZvhvGhP" rel="nofollow noopener" target="_blank" title="https://mars.nasa.gov/mars2020/mission/where-is-the-rover/"><span class="tco-ellipsis"></span><span class="invisible">https://</span><span class="js-display-url">mars.nasa.gov/mars2020/missi</span><span class="invisible">on/where-is-the-rover/</span><span class="tco-ellipsis"><span class="invisible"> </span>…</span></a><a class="twitter-timeline-link u-hidden" data-pre-embedded="true" dir="ltr" href="https://t.co/DRHHZpRI3f">pic.twitter.com/DRHH

In [15]:
# find the latest tweet con't
mars_weather_first = latest_weather.find('p', class_='js-tweet-text').text
mars_weather_first

'The #Mars2020 rover gets her wheels from the pit crew with PhDs\nhttps://mars.nasa.gov/mars2020/mission/where-is-the-rover/\xa0…pic.twitter.com/DRHHZpRI3f'

In [16]:
# the lastest tweet did not contain weather data so find all the tweets
latest_weather_all = twitter_soup.find_all('div', class_='js-tweet-text-container')
print(latest_weather_all)

[<div class="js-tweet-text-container">
<p class="TweetTextSize TweetTextSize--normal js-tweet-text tweet-text" data-aria-label-part="0" lang="en">The <a class="twitter-hashtag pretty-link js-nav" data-query-source="hashtag_click" dir="ltr" href="/hashtag/Mars2020?src=hash"><s>#</s><b>Mars2020</b></a> rover gets her wheels from the pit crew with PhDs
<a class="twitter-timeline-link" data-expanded-url="https://mars.nasa.gov/mars2020/mission/where-is-the-rover/" dir="ltr" href="https://t.co/525ZvhvGhP" rel="nofollow noopener" target="_blank" title="https://mars.nasa.gov/mars2020/mission/where-is-the-rover/"><span class="tco-ellipsis"></span><span class="invisible">https://</span><span class="js-display-url">mars.nasa.gov/mars2020/missi</span><span class="invisible">on/where-is-the-rover/</span><span class="tco-ellipsis"><span class="invisible"> </span>…</span></a><a class="twitter-timeline-link u-hidden" data-pre-embedded="true" dir="ltr" href="https://t.co/DRHHZpRI3f">pic.twitter.com/DRH

In [17]:
# make a list of all tweets and store in a list
weather_list = []
for weather in latest_weather_all:
    mars_weather = weather.find('p', class_='js-tweet-text').text
    weather_list.append(mars_weather)
    
weather_list

['The #Mars2020 rover gets her wheels from the pit crew with PhDs\nhttps://mars.nasa.gov/mars2020/mission/where-is-the-rover/\xa0…pic.twitter.com/DRHHZpRI3f',
 'InSight sol 219 (2019-07-09) low -100.3ºC (-148.5ºF) high -24.9ºC (-12.8ºF)\nwinds from the SE at 4.5 m/s (10.0 mph) gusting to 16.1 m/s (36.1 mph)\npressure at 7.60 hPapic.twitter.com/zJkUifSPvt',
 'InSight sol 218 (2019-07-08) low -100.2ºC (-148.4ºF) high -26.1ºC (-15.1ºF)\nwinds from the SE at 4.6 m/s (10.2 mph) gusting to 16.0 m/s (35.8 mph)\npressure at 7.60 hPa',
 'InSight sol 216 (2019-07-06) low -102.5ºC (-152.5ºF) high -24.9ºC (-12.8ºF)\nwinds from the SSE at 4.6 m/s (10.2 mph) gusting to 15.8 m/s (35.4 mph)\npressure at 7.60 hPapic.twitter.com/fwGott1wRL',
 'InSight sol 215 (2019-07-05) low -101.7ºC (-151.0ºF) high -25.0ºC (-12.9ºF)\nwinds from the SSE at 4.2 m/s (9.3 mph) gusting to 14.6 m/s (32.6 mph)\npressure at 7.60 hPa',
 'InSight sol 214 (2019-07-04) low -101.2ºC (-150.1ºF) high -24.7ºC (-12.4ºF)\nwinds from th

In [18]:
# this is the lasest weather related tweet
#print(weather_list[1])

In [19]:
#list comprehesion to search the list for the first weather related tweet
mars_weather = [x for x in weather_list if 'InSight' in x][0]
mars_weather

'InSight sol 219 (2019-07-09) low -100.3ºC (-148.5ºF) high -24.9ºC (-12.8ºF)\nwinds from the SE at 4.5 m/s (10.0 mph) gusting to 16.1 m/s (36.1 mph)\npressure at 7.60 hPapic.twitter.com/zJkUifSPvt'

## Mars Facts

In [20]:
# * Visit the Mars Facts webpage [here](https://space-facts.com/mars/) and use Pandas to scrape the table containing facts about the planet including Diameter, Mass, etc.
# * Use Pandas to convert the data to a HTML table string.
mars_facts_url = 'https://space-facts.com/mars/'
mars_facts_tables = pd.read_html(mars_facts_url)
mars_facts_tables

[  Mars - Earth Comparison             Mars            Earth
 0               Diameter:         6,779 km        12,742 km
 1                   Mass:  6.39 × 10^23 kg  5.97 × 10^24 kg
 2                  Moons:                2                1
 3      Distance from Sun:   227,943,824 km   149,598,262 km
 4         Length of Year:   687 Earth days      365.24 days
 5            Temperature:    -153 to 20 °C      -88 to 58°C,
                       0                              1
 0  Equatorial Diameter:                       6,792 km
 1       Polar Diameter:                       6,752 km
 2                 Mass:  6.39 × 10^23 kg (0.11 Earths)
 3                Moons:            2 (Phobos & Deimos)
 4       Orbit Distance:       227,943,824 km (1.38 AU)
 5         Orbit Period:           687 days (1.9 years)
 6  Surface Temperature:                   -87 to -5 °C
 7         First Record:              2nd millennium BC
 8          Recorded By:           Egyptian astronomers]

In [21]:
# select the second table, set the column headers, reset index to 'Description'
mars_facts_tables_df = mars_facts_tables[1]
mars_facts_tables_df.columns = ['Description', 'Value']
mars_facts_tables_df.set_index('Description', inplace=True)
mars_facts_tables_df

,Value
Description,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.39 × 10^23 kg (0.11 Earths)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.38 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-87 to -5 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers


In [22]:
# export an html file of mars facts
mars_facts = mars_facts_tables_df.to_html('mars_facts.html')
mars_facts

## Mars Hemisphere Images

In [23]:
# * Visit the USGS Astrogeology site [here](https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars) to obtain high resolution images for each of Mars' hemispheres.
# * You will need to click each of the links to the hemispheres in order to find the image url to the full resolution image.
# * Save both the image url string for the full resolution hemisphere image, and the Hemisphere title containing the hemisphere name. Use a Python dictionary to store the data using the keys `img_url` and `title`.
# * Append the dictionary with the image url string and the hemisphere title to a list. This list will contain one dictionary for each hemisphere.

mars_hemisphere_url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(mars_hemisphere_url)
mars_hemisphere_html = browser.html
mars_hemisphere_soup = bs(mars_hemisphere_html, 'lxml')


list_of_mars_hemispheres = mars_hemisphere_soup.find_all('div', class_='item')
#print(list_of_mars_hemispheres)

In [24]:
# list to store all dictionary data
mars_hemisphere_image_urls = []

#loop through images and click high res image
base_url = 'https://astrogeology.usgs.gov'

for image in list_of_mars_hemispheres:
    hemi_dict = {}
    mars_image_hemi = image.find('a', class_='itemLink product-item')
    link = base_url + mars_image_hemi['href']
    browser.visit(link)
    
    #pause execution for 1 second to prevent errors
    time.sleep(1)
    
    #print(link)
    
    mars_hemisphere_html_loop = browser.html
    mars_hemisphere_html_loop_soup = bs(mars_hemisphere_html_loop, 'lxml')
    
    #collect image titles
    image_title = mars_hemisphere_html_loop_soup.find('div', class_='content').find('h2', class_='title').text
    #print(image_title)
    #save as dictionary with key 'title'
    hemi_dict['title'] = image_title
    
    #collect image links to high res images
    image_link = mars_hemisphere_html_loop_soup.find('div', class_='downloads').find('a')['href']
    #print(image_link)
    #save as dictionary with key 'img_url'
    hemi_dict['img_url'] = image_link   
    
    # append dictionaries to list
    mars_hemisphere_image_urls.append(hemi_dict)
    
mars_hemisphere_image_urls    

[{'title': 'Cerberus Hemisphere Enhanced',
  'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'},
 {'title': 'Schiaparelli Hemisphere Enhanced',
  'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'},
 {'title': 'Syrtis Major Hemisphere Enhanced',
  'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg'},
 {'title': 'Valles Marineris Hemisphere Enhanced',
  'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}]